# ENTORNO, LOGGING Y DEPENDENCIAS (KAGGLE)

# Integrantes:


* Yesid Castelblanco
* Albin Rivera


El presente notebook está orientado al desarrollo de un flujo completo de clasificación de texto en español empleando el modelo BERT dentro del ecosistema de Hugging Face Transformers. Su diseño está pensado para ejecutarse en entornos de experimentación como Kaggle, integrando desde la configuración del entorno y carga de datos hasta la optimización de hiperparámetros y el entrenamiento final del modelo.

La estructura se encuentra organizada en las siguientes secciones:


* Preparación del entorno y dependencias.
* Importación de librerías y componentes del pipeline.
* Carga y exploración de datos en español.
* Tokenización y construcción del modelo BERT.
* Definición de métricas de evaluación (accuracy y F1).
* Búsqueda de hiperparámetros con Optuna.
* Entrenamiento final utilizando la mejor configuración encontrada.
  

# OBJETIVO

Implementar un pipeline reproducible y optimizado de clasificación de texto en español con BERT, que integre la carga de datos, el preprocesamiento mediante tokenización, la definición de métricas, la optimización de hiperparámetros y el entrenamiento final, garantizando un modelo con buen desempeño evaluado mediante accuracy y F1-score.

# DESARROLLO DEL TALLER

A continuacion, se prepara el entorno de ejecución del notebook asegurando su compatibilidad en Kaggle, Google Colab o localmente. Para ello, se configura un sistema de logging que centraliza mensajes en un archivo y silencia salidas innecesarias de librerías como TensorFlow. Además, detecta el entorno en que se ejecuta el script y ajusta variables en consecuencia, definiendo directorios de caché persistente en Kaggle para optimizar descargas de Hugging Face. De igual forma se implementan funciones que verifican e instalan versiones específicas de paquetes clave (transformers, datasets, accelerate, optuna, entre otros), garantizando estabilidad y reproducibilidad. También se descargan recursos lingüísticos de NLTK en español (stopwords y WordNet), y se aplican parámetros de optimización para el procesamiento eficiente de tokens y operaciones matriciales en PyTorch. En síntesis, este bloque asegura que todas las dependencias y configuraciones necesarias estén listas antes de iniciar el pipeline de clasificación de texto.

In [1]:
# ==== DEPLOY KAGGLE: LOGGING, ENV, INSTALLS ====
import os, sys, subprocess, warnings, logging, pkg_resources

# Logging
logging.basicConfig(
    filename='training.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
logging.getLogger("xla").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger().setLevel(logging.CRITICAL)
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    sys.stderr = open('/dev/null', 'w')

IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
IN_COLAB = False
if not IN_KAGGLE:
    try:
        import google.colab  # noqa
        IN_COLAB = True
        print("Ejecutando en Google Colab..."); logger.info("Colab")
    except ImportError:
        print("Ejecutando localmente..."); logger.info("Local")
else:
    print("Ejecutando en Kaggle..."); logger.info("Kaggle")

def install_package(spec, extra_index_url=None):
    cmd = [sys.executable, "-m", "pip", "install", "-q", "--no-warn-conflicts", "--upgrade"]
    if extra_index_url:
        cmd += ["--index-url", extra_index_url]
    cmd += spec.split()
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def ver(pkg):
    try:
        return pkg_resources.get_distribution(pkg).version
    except pkg_resources.DistributionNotFound:
        return None

def ensure(pkg, target_version=None, spec=None, extra_index_url=None):
    """
    - pkg: nombre para verificar (e.g., 'transformers')
    - target_version: '4.45.2' para comprobar igualdad; si None solo verifica existencia
    - spec: string completo pip (e.g., 'transformers==4.45.2 datasets==3.0.1')
    """
    v = ver(pkg)
    if (v is None) or (target_version and v != target_version):
        install_package(spec if spec else pkg, extra_index_url)

# Caché HF persistente en Kaggle
if IN_KAGGLE:
    cache_dir = "/kaggle/working/hf_cache"
    os.makedirs(cache_dir, exist_ok=True)
    os.environ["HF_HOME"] = cache_dir
    os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
    os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")

# --- Paquetes clave (NO forzamos torch por defecto) ---
# Nota: Kaggle ya trae torch+CUDA funcional. Solo si necesitas forzar:
FORZAR_TORCH = False  # cambia a True si realmente lo necesitas

if (IN_COLAB or IN_KAGGLE) and FORZAR_TORCH:
    # CUDA 11.8 (suele ser compatible con P100 en Kaggle)
    ensure("torch", target_version="2.3.1",  # versión estable conocida; ajusta si lo requieres
           spec="torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1",
           extra_index_url="https://download.pytorch.org/whl/cu118")

# Core HF y ciencia de datos
ensure("transformers", "4.45.2", spec="transformers==4.45.2")
ensure("datasets", "3.0.1", spec="datasets==3.0.1")
ensure("accelerate", "0.34.2", spec="accelerate==0.34.2")
ensure("evaluate", "0.4.3", spec="evaluate==0.4.3")
ensure("sentence-transformers", "3.2.1", spec="sentence-transformers==3.2.1")
ensure("peft", "0.13.2", spec="peft==0.13.2")
ensure("nltk", "3.9.1", spec="nltk==3.9.1")
ensure("nlpaug", "1.1.11", spec="nlpaug==1.1.11")
ensure("optuna", "3.6.1", spec="optuna==3.6.1")

# (Opcionales, por si tu pipeline los usa)
ensure("numpy", "1.26.4", spec="numpy==1.26.4")
ensure("pandas", "2.2.2", spec="pandas==2.2.2")
ensure("pyarrow", "15.0.2", spec="pyarrow==15.0.2")

# Verificación rápida
try:
    import torch, transformers, datasets, evaluate, accelerate, optuna, nltk
except ImportError as e:
    logger.error(f"Import error: {e}")
    raise

# NLTK y entorno
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
# Este tagger inglés no es necesario para ES; lo dejamos por compat:
try:
    nltk.download('averaged_perceptron_tagger_eng', quiet=True)
except Exception:
    pass

stop_words = set(stopwords.words('spanish'))

# Silencio y precisión
from transformers.utils import logging as hf_logging
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
hf_logging.set_verbosity_error()
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/lib/cuda"
try:
    torch.set_float32_matmul_precision('medium')
except Exception:
    pass

print("Entorno listo ✅")
print("Versions -> torch:", torch.__version__,
      "| transformers:", transformers.__version__,
      "| datasets:", datasets.__version__,
      "| accelerate:", accelerate.__version__,
      "| evaluate:", evaluate.__version__)



Ejecutando en Kaggle...


2025-09-13 19:54:45.002767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757793285.027247     185 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757793285.034467     185 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Entorno listo ✅
Versions -> torch: 2.6.0+cu124 | transformers: 4.45.2 | datasets: 3.0.1 | accelerate: 0.34.2 | evaluate: 0.4.3


# 1) IMPORTS DEL PIPELINE

A continuacion se implementan las librerías requeridas para la construcción del flujo de clasificación de texto. Se importan módulos de Hugging Face para el preprocesamiento y entrenamiento del modelo (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding) y la utilidad load_dataset para acceder a conjuntos de datos. Además, se incluyen librerías de apoyo como numpy, evaluate y random, que permiten manejar cálculos numéricos, métricas y reproducibilidad en los experimentos. Finalmente, se define la variable DEVICE que detecta si hay una GPU disponible para ejecutar el entrenamiento en CUDA, lo cual optimiza el rendimiento, o en caso contrario se recurre a la CPU. En resumen, este bloque inicializa las herramientas clave que sostendrán las etapas de carga, tokenización, entrenamiento y evaluación del modelo.

In [2]:
# ============================
# 1) IMPORTS DEL PIPELINE
# ============================
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding
)
import numpy as np
import evaluate
import random

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Dispositivo:", DEVICE)


Dispositivo: cuda


# 2) CARGA DE DATOS (ESPAÑOL)

Aqui se lleva a cabo la carga de datos donde se utiliza el conjunto de reseñas en español de Amazon Reviews Multi disponible en Hugging Face. Esta base de datos contiene opiniones de clientes en español, donde cada texto está asociado a una etiqueta de sentimiento representada en escala de 0 a 4, que equivale a calificaciones de 1 a 5 estrellas (desde muy negativa hasta muy positiva). El código descarga el dataset desde "SetFit/amazon_reviews_multi_es" y lo organiza en las particiones estándar: entrenamiento, validación y prueba. Para facilitar los experimentos y optimizar el uso de recursos (tiempo y memoria de GPU), se implementa un submuestreo que reduce la cantidad de ejemplos en este caso de 8.000 para entrenamiento y 2.000 para validación, manteniendo la totalidad del conjunto de prueba. De esta manera, se asegura que el pipeline pueda probarse de forma ágil sin comprometer la calidad del entrenamiento. 

In [3]:
# ============================
# 2) CARGA DE DATOS (ESPAÑOL)
# ============================
# Dataset de reseñas en español (etiquetas 0..4 ~ 1..5 estrellas)
ds = load_dataset("SetFit/amazon_reviews_multi_es", cache_dir=os.environ.get("HF_DATASETS_CACHE", None))
print(ds)
print(ds["train"][0])

# (Opcional) Submuestreo para búsq. rápida de HP; ajusta según tiempo/VRAM:
SUBSET_TRAIN = 8000    # puedes subir a 20000 si la P100 te da margen
SUBSET_VAL   = 2000
train_ds = ds["train"].shuffle(seed=42).select(range(min(SUBSET_TRAIN, len(ds["train"]))))
val_ds   = ds["validation"].shuffle(seed=42).select(range(min(SUBSET_VAL, len(ds["validation"]))))
test_ds  = ds["test"]


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})
{'id': 'es_0491108', 'text': 'Nada bueno se me fue ka pantalla en menos de 8 meses y no he recibido respuesta del fabricante', 'label': 0, 'label_text': '0'}


# 3) TOKENIZACIÓN Y MODELO

Para esta etapa,se emplea como base el modelo BETO, una versión de BERT entrenada específicamente para español. Con su tokenizer, los textos del dataset se transforman en secuencias de tokens con truncamiento a un máximo de 256, aplicándose sobre los conjuntos de entrenamiento, validación y prueba. Al mismo tiempo, se conserva únicamente la información relevante de cada registro (text y label). Se calcula el número de clases a partir de las etiquetas y se define un data collator con padding dinámico, lo que permite que los lotes tengan la misma longitud y optimicen el uso de GPU. En síntesis, este bloque estandariza los datos textuales y establece los insumos para que BETO pueda entrenarse en la clasificación de reseñas en español.

In [4]:
# ============================
# 3) TOKENIZACIÓN Y MODELO
# ============================
MODEL_NAME = "dccuchile/bert-base-spanish-wwm-cased"  # BETO
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, cache_dir=os.environ.get("TRANSFORMERS_CACHE", None))

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

tok_train = train_ds.map(tokenize, batched=True, remove_columns=[c for c in train_ds.column_names if c not in ["text", "label"]])
tok_val   = val_ds.map(tokenize, batched=True, remove_columns=[c for c in val_ds.column_names if c not in ["text", "label"]])
tok_test  = test_ds.map(tokenize, batched=True, remove_columns=[c for c in test_ds.column_names if c not in ["text", "label"]])

num_labels = len(set(ds["train"]["label"]))
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# 4) MÉTRICAS (ACC y F1 weighted)

Se define las métricas de evaluación para el modelo, las cuales son accuracy (precisión global) y F1-score con promedio ponderado, que considera el desempeño de cada clase en función de su peso en el dataset. La función compute_metrics toma como entrada las predicciones del modelo (logits) y las etiquetas reales, convierte los logits en clases predichas mediante argmax, y luego calcula los valores de ambas métricas. De esta manera, se obtiene una evaluación equilibrada, donde la accuracy mide qué tan a menudo el modelo acierta en general, mientras que el F1 ponderado refleja la calidad de la clasificación en datasets con múltiples clases, ajustando el impacto de clases más frecuentes y menos frecuentes. 

In [5]:
# ============================
# 4) MÉTRICAS (ACC y F1 weighted)
# ============================
acc = evaluate.load("accuracy")
f1w = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1w.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }


Este bloque ejecuta la limpieza de directorios temporales y de salida en el entorno de Kaggle. El primer comando (!rm -rf /kaggle/working/hf_cache) elimina la caché de Hugging Face almacenada durante ejecuciones previas, evitando que queden restos de modelos o datasets que puedan generar conflictos. El segundo (!rm -rf outputs optuna_search) borra las carpetas de resultados y de búsquedas de hiperparámetros de Optuna, garantizando que las siguientes corridas del notebook empiecen desde cero y con un entorno limpio. En resumen, este paso asegura que el entrenamiento y la optimización se realicen sin residuos de ejecuciones anteriores, lo cual favorece la reproducibilidad y la consistencia de los experimentos.

In [10]:
!rm -rf /kaggle/working/hf_cache
!rm -rf outputs optuna_search


# 5) BÚSQUEDA DE HIPERPARÁMETROS (OPTUNA)

Aqui se realiza la optimización de hiperparámetros con Optuna para mejorar el desempeño del modelo BETO en la clasificación de reseñas. Se define una función de métricas (accuracy y F1 ponderado), una inicialización del modelo y un conjunto de argumentos de entrenamiento simplificados. Para agilizar el proceso, se emplea un subset reducido de datos. Optuna explora combinaciones de tasa de aprendizaje, batch size, número de épocas y weight decay en varias pruebas, y al final se reporta la mejor configuración encontrada.

In [11]:
# ============================
# 5) BÚSQUEDA DE HIPERPARÁMETROS (OPTUNA)
# ============================
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_labels,
        cache_dir=os.environ.get("TRANSFORMERS_CACHE", None)
    )

# Argumentos base: no guardar checkpoints, carpeta temporal
search_args = TrainingArguments(
    output_dir="/kaggle/temp/optuna_search",
    evaluation_strategy="epoch",
    save_strategy="no",
    load_best_model_at_end=False,
    report_to="none",
    fp16=True,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=50,
    seed=42
)

# Subset para búsqueda rápida
tok_train_small = tok_train.shuffle(seed=42).select(range(2000))
tok_val_small   = tok_val.shuffle(seed=42).select(range(500))

search_trainer = Trainer(
    model_init=model_init,
    args=search_args,
    train_dataset=tok_train_small,
    eval_dataset=tok_val_small,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

best_run = search_trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=6,  # baja a 6-8 para no llenar disco
    hp_space=lambda trial: {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 4),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
    }
)

print("Mejor ejecución (Optuna):", best_run)


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'eval_loss': 1.2298344373703003, 'eval_accuracy': 0.434, 'eval_f1': 0.39799723533216663, 'eval_runtime': 1.6944, 'eval_samples_per_second': 295.093, 'eval_steps_per_second': 18.886, 'epoch': 0.9841269841269841}
{'loss': 1.3366, 'grad_norm': 4.5296406745910645, 'learning_rate': 1.0213608806609128e-05, 'epoch': 1.5873015873015874}
{'eval_loss': 1.1346220970153809, 'eval_accuracy': 0.478, 'eval_f1': 0.467624630213346, 'eval_runtime': 1.6975, 'eval_samples_per_second': 294.556, 'eval_steps_per_second': 18.852, 'epoch': 2.0}
{'eval_loss': 1.129022240638733, 'eval_accuracy': 0.484, 'eval_f1': 0.46446599113111553, 'eval_runtime': 1.6973, 'eval_samples_per_second': 294.592, 'eval_steps_per_second': 18.854, 'epoch': 2.9523809523809526}
{'train_runtime': 77.8079, 'train_samples_per_second': 77.113, 'train_steps_per_second': 1.195, 'train_loss': 1.1881442736553889, 'epoch': 2.9523809523809526}
{'loss': 1.3493, 'grad_norm': 6.630311012268066, 'learning_rate': 3.0175304438273315e-05, 'epoch': 0.8}

La búsqueda de hiperparámetros con Optuna arrojó múltiples ejecuciones con diferentes combinaciones de tasa de aprendizaje, tamaño de batch, número de épocas y weight decay, registrando métricas de exactitud y F1 ponderado en cada caso. Tras las pruebas, la mejor configuración encontrada fue un learning rate cercano a 3.3e-5, un batch size de 32, 4 épocas de entrenamiento y un weight decay de aproximadamente 0.17, parámetros que permitieron alcanzar el mejor equilibrio entre accuracy y F1-score en la tarea de clasificación de reseñas en español.

# 6) ENTRENAMIENTO FINAL CON LOS MEJORES HP

Aqui se lleva a cabo el entrenamiento final del modelo, utilizando los hiperparámetros óptimos hallados en la búsqueda con Optuna. Primero, se aplican estos valores directamente a los argumentos del trainer, garantizando que el proceso de ajuste se realice con la mejor configuración encontrada. Luego, se actualiza la función de métricas (compute_metrics) para que opere con la versión definida previamente. Finalmente, se ejecuta el entrenamiento completo sobre todo el dataset, no solo con subsets como en la fase de prueba, con lo cual el modelo aprovecha la totalidad de los datos disponibles para aprender y lograr un mejor rendimiento en la tarea de clasificación de reseñas en español. 

In [13]:
# ============================
# 6) ENTRENAMIENTO FINAL CON LOS MEJORES HP
# ============================

# Aplicar hiperparámetros óptimos al trainer
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

print("Hiperparámetros finales:", trainer.args)
logger.info(f"Final Training Args: {trainer.args}")

# Reemplazar compute_metrics por la versión sin evaluate
trainer.compute_metrics = compute_metrics

# Entrenamiento final completo en todo el dataset
trainer.train()



Hiperparámetros finales: TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=True,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_ob

TrainOutput(global_step=500, training_loss=0.8705581359863281, metrics={'train_runtime': 417.9748, 'train_samples_per_second': 76.56, 'train_steps_per_second': 1.196, 'train_loss': 0.8705581359863281, 'epoch': 4.0})

# 7) EVALUACIÓN EN TEST

Aqui se realiza la evaluación final del modelo en el conjunto de prueba (test) y se visualiza el resultado de entrenamiento. Primero, se ejecuta trainer.evaluate(tok_test) para medir el desempeño del modelo sobre datos nunca vistos, obteniendo las métricas finales de exactitud y F1 ponderado. Estas métricas son impresas y registradas en los logs, lo que permite validar de manera objetiva la capacidad de generalización del modelo. Luego, se guarda el modelo entrenado junto con su tokenizer en la carpeta outputs/best_model, lo que facilita reutilizarlo en tareas de inferencia o despliegue, posteriormente sin necesidad de reentrenar. En síntesis, este bloque valida el rendimiento definitivo alcanzado y almacena el modelo listo para tareas de inferencia o despliegue.

In [14]:
# ============================
# 7) EVALUACIÓN EN TEST
# ============================
metrics = trainer.evaluate(tok_test)
print("Métricas finales en test:", metrics)
logger.info(f"Test metrics: {metrics}")

# Guardar el mejor modelo
save_dir = "outputs/best_model"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Modelo guardado en: {save_dir}")


{'eval_loss': 1.0524392127990723, 'eval_accuracy': 0.539, 'eval_f1': 0.5451025028597957, 'eval_runtime': 17.0144, 'eval_samples_per_second': 293.868, 'eval_steps_per_second': 18.396, 'epoch': 4.0}
Métricas finales en test: {'eval_loss': 1.0524392127990723, 'eval_accuracy': 0.539, 'eval_f1': 0.5451025028597957, 'eval_runtime': 17.0144, 'eval_samples_per_second': 293.868, 'eval_steps_per_second': 18.396, 'epoch': 4.0}
Modelo guardado en: outputs/best_model


Los resultados de la evaluación final en el conjunto de prueba muestran que el modelo alcanzó una exactitud (accuracy) de 0.539 y un F1 ponderado de 0.545, con una pérdida en validación de aproximadamente 1.05. Estos valores indican un rendimiento moderado, es decir que el modelo logra clasificar correctamente algo más de la mitad de los ejemplos y mantiene un equilibrio aceptable entre precisión y cobertura en las distintas clases.

# 8) PROBAR EL MODELO CON TEXTO NUEVO

Finalmente se implementa la fase de inferencia del pipeline, es decir, probar el modelo ya entrenado con textos nuevos. Para ello, se carga el modelo y el tokenizer desde la carpeta outputs/best_model usando la función pipeline de Hugging Face en modo de clasificación de texto. Luego, se definen algunos ejemplos en español que representan reseñas positivas, negativas y neutras. El modelo procesa estos textos y genera predicciones de clase (LABEL_X) junto con un puntaje de confianza. Posteriormente, estas etiquetas numéricas se convierten en un formato más intuitivo de estrellas (de 1 a 5), lo que facilita la interpretación del resultado. En síntesis, este bloque valida de forma práctica el funcionamiento del modelo entrenado, mostrando cómo traduce reseñas escritas en español en calificaciones de estrellas con su respectivo nivel de confianza.

In [15]:
# ============================
# 8) PROBAR EL MODELO CON TEXTO NUEVO
# ============================

from transformers import pipeline

# Ruta del modelo guardado
MODEL_DIR = "outputs/best_model"

# Cargar pipeline de clasificación
classifier = pipeline("text-classification", model=MODEL_DIR, tokenizer=MODEL_DIR, device=0)

# Ejemplos de prueba
ejemplos = [
    "Este producto es excelente, llegó rápido y funciona muy bien",
    "El paquete vino roto y el producto no sirve",
    "Está bien, cumple pero nada especial"
]

# Obtener predicciones
predicciones = classifier(ejemplos)

# Mapear LABEL_X a estrellas
label2stars = {0: "⭐ (1 estrella)", 1: "⭐⭐", 2: "⭐⭐⭐", 3: "⭐⭐⭐⭐", 4: "⭐⭐⭐⭐⭐"}

# Mostrar resultados
for texto, pred in zip(ejemplos, predicciones):
    label_num = int(pred['label'].split("_")[-1])
    print(f"Texto: {texto}\n→ Predicción: {label2stars[label_num]} (confianza: {pred['score']:.2f})\n")


Texto: Este producto es excelente, llegó rápido y funciona muy bien
→ Predicción: ⭐⭐⭐⭐⭐ (confianza: 0.75)

Texto: El paquete vino roto y el producto no sirve
→ Predicción: ⭐ (1 estrella) (confianza: 0.86)

Texto: Está bien, cumple pero nada especial
→ Predicción: ⭐⭐⭐ (confianza: 0.69)



Los resultados de prueba muestran que el modelo logra identificar adecuadamente el sentimiento de los textos nuevos; a la reseña positiva “Este producto es excelente, llegó rápido y funciona muy bien” le asigna 5 estrellas con una confianza del 0.75; a la reseña negativa “El paquete vino roto y el producto no sirve” la clasifica con 1 estrella y una alta confianza de 0.86; y a la reseña más neutra “Está bien, cumple pero nada especial” la ubica en 3 estrellas con confianza de 0.69. Esto evidencia que el modelo puede diferenciar entre opiniones positivas, negativas y moderadas, generando predicciones coherentes con el sentido general de cada texto.

# 9) CONCLUSIONES

* La búsqueda de hiperparámetros con Optuna permitió encontrar una configuración adecuada (lr ≈ 3.3e-5, batch size 32, 4 épocas, weight decay ≈ 0.17), la cual se aplicó en el entrenamiento final para maximizar las métricas de desempeño.
* En el conjunto de prueba, el modelo alcanzó un accuracy de 0.539 y un F1 ponderado de 0.545, lo que demuestra que logra una clasificación moderada de las reseñas, cumpliendo con el objetivo de obtener un modelo funcional aunque con margen de mejora en la generalización.
* Las pruebas con ejemplos nuevos evidenciaron que el modelo es capaz de asignar predicciones coherentes con el sentimiento de los textos, diferenciando entre opiniones positivas, negativas y neutras, lo que valida su aplicabilidad en escenarios reales.
* El modelo final fue guardado junto con su tokenizer, quedando listo para ser reutilizado en tareas de inferencia o integrarse en aplicaciones prácticas de análisis de reseñas, cerrando el ciclo planteado en el objetivo inicial.